In [3]:
!CUDA_VISIBLE_DEVICES=0,1 \
python train.py \
--dataroot /home/user9/us_train \
--testdataroot /home/user9/us_test \
--name test \
--n_epochs 200 \
--gpu_ids 1 \
--input_nc 1 \
--output_nc 1 \
--batch_size 4 \
--phase train \
--is_mtl \
--dataset_mode unaligned2

----------------- Options ---------------
               batch_size: 4                             	[default: 1]
                    beta1: 0.5                           
          checkpoints_dir: ../checkpoints                
           continue_train: False                         
                crop_size: 256                           
                   d_norm: basic                         
                data_norm: basic                         
                data_type: all                           
                 dataroot: /home/user9/us_train          	[default: None]
             dataset_mode: unaligned2                    	[default: unaligned]
                direction: AtoB                          
                    disLR: False                         
              display_env: main                          
             display_freq: 400                           
               display_id: 1                             
            display_ncols: 4          

In [3]:
!nohup CUDA_VISIBLE_DEVICES=0,1 python /home/user9/USenhance23_POAS-main/train.py --dataroot /home/user9/us_train --testdataroot /home/user9/us_test --name test --n_epochs 100 --checkpoints_dir /home/user9/checkpoints --gpu_ids 1 \
--input_nc 1 \
--output_nc 1 \
--batch_size 4 \
--phase train \
--is_mtl \
--dataset_mode unaligned2 &

OSError: Background processes not supported.

In [2]:
!CUDA_VISIBLE_DEVICES=0,1 \
python test.py \
--results_dir /home/user9/us_test/result \
--name test \
--model_suffix _A \
--dataset_mode unaligned2 \
--gpu_ids 0 \
--input_nc 1 \
--output_nc 1 \
--checkpoints_dir /home/user9/checkpoint \
--batch_size 24 \
--phase test \
--testdataroot /home/user9/us_test \
--dataroot /home/user9/us_test \
--serial_batches 1 \
--is_mtl 

----------------- Options ---------------
             aspect_ratio: 1.0                           
               batch_size: 24                            	[default: 1]
          checkpoints_dir: /home/user9/checkpoint        	[default: ../checkpoints]
                crop_size: 256                           
                   d_norm: basic                         
                data_norm: basic                         
                data_type: all                           
                 dataroot: /home/user9/us_test           	[default: None]
             dataset_mode: unaligned2                    	[default: single]
                direction: AtoB                          
                    disLR: False                         
          display_winsize: 256                           
                    epoch: latest                        
                     eval: False                         
                  gpu_ids: 0                             
               

In [8]:
import os
import numpy as np
import cv2
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr

def calculate_lncc(image1, image2):
    """
    计算局部归一化互相关 (LNCC)
    """
    # 将图像转换为浮点型
    image1 = image1.astype(np.float32)
    image2 = image2.astype(np.float32)
    
    # 计算局部均值
    mean1 = cv2.blur(image1, (5, 5))  # 使用5x5的窗口
    mean2 = cv2.blur(image2, (5, 5))
    
    # 计算局部方差和协方差
    var1 = cv2.blur((image1 - mean1) ** 2, (5, 5))
    var2 = cv2.blur((image2 - mean2) ** 2, (5, 5))
    cov = cv2.blur((image1 - mean1) * (image2 - mean2), (5, 5))
    
    # 避免除以零
    var1[var1 == 0] = 1e-10
    var2[var2 == 0] = 1e-10
    
    # 计算LNCC
    lncc = cov / (np.sqrt(var1) * np.sqrt(var2) + 1e-10)
    
    # 返回平均LNCC
    return np.mean(lncc)

def calculate_metrics(real_dir, generated_dir):
    """
    计算真实图像和生成图像之间的SSIM、PSNR和LNCC
    """
    ssim_scores = []
    psnr_scores = []
    lncc_scores = []
    
    # 遍历真实图像目录
    for filename in os.listdir(real_dir):
        if filename.endswith(".png") or filename.endswith(".jpg"):
            real_path = os.path.join(real_dir, filename)
            generated_path = os.path.join(generated_dir, filename)
            
            # 读取图像
            real_image = cv2.imread(real_path, cv2.IMREAD_GRAYSCALE)
            generated_image = cv2.imread(generated_path, cv2.IMREAD_GRAYSCALE)
            
            # 确保图像大小一致
            if real_image.shape != generated_image.shape:
                generated_image = cv2.resize(generated_image, (real_image.shape[1], real_image.shape[0]))
            
            # 计算SSIM
            ssim_score = ssim(real_image, generated_image, data_range=real_image.max() - real_image.min())
            ssim_scores.append(ssim_score)
            
            # 计算PSNR
            psnr_score = psnr(real_image, generated_image, data_range=real_image.max() - real_image.min())
            psnr_scores.append(psnr_score)
            
            # 计算LNCC
            lncc_score = calculate_lncc(real_image, generated_image)
            lncc_scores.append(lncc_score)
    
    # 计算平均指标
    avg_ssim = np.mean(ssim_scores)
    avg_psnr = np.mean(psnr_scores)
    avg_lncc = np.mean(lncc_scores)
    
    return avg_ssim, avg_psnr, avg_lncc

# 定义真实图像和生成图像的目录
real_dir = "/home/user9/us_test/high_quality"
generated_dir = "/home/user9/us_test/result"

# 计算指标
avg_ssim, avg_psnr, avg_lncc = calculate_metrics(real_dir, generated_dir)

# 输出结果
print(f"Average SSIM: {avg_ssim:.4f}")
print(f"Average PSNR: {avg_psnr:.4f}")
print(f"Average LNCC: {avg_lncc:.4f}")

Average SSIM: 0.2044
Average PSNR: 13.9686
Average LNCC: 0.0041


In [4]:
import os
import numpy as np
import cv2
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr
from collections import defaultdict

def calculate_lncc(image1, image2):
    """
    计算局部归一化互相关 (LNCC)
    """
    image1 = image1.astype(np.float32)
    image2 = image2.astype(np.float32)
    
    mean1 = cv2.blur(image1, (5, 5))
    mean2 = cv2.blur(image2, (5, 5))
    
    var1 = cv2.blur((image1 - mean1) ** 2, (5, 5))
    var2 = cv2.blur((image2 - mean2) ** 2, (5, 5))
    cov = cv2.blur((image1 - mean1) * (image2 - mean2), (5, 5))
    
    var1[var1 == 0] = 1e-10
    var2[var2 == 0] = 1e-10
    
    lncc = cov / (np.sqrt(var1) * np.sqrt(var2) + 1e-10)
    return np.mean(lncc)

def calculate_metrics_by_class(real_dir, generated_dir):
    """
    按类别计算真实图像和生成图像之间的SSIM、PSNR和LNCC
    """
    # 初始化字典存储每个类别的指标
    class_metrics = defaultdict(lambda: {
        'ssim': [],
        'psnr': [],
        'lncc': []
    })
    
    # 遍历真实图像目录
    for filename in os.listdir(real_dir):
        if filename.endswith((".png", ".jpg")):
            # 提取类别名称（假设文件名格式为"类别_编号.扩展名"）
            class_name = filename.split('_')[0]
            
            real_path = os.path.join(real_dir, filename)
            generated_path = os.path.join(generated_dir, filename)
            
            # 检查生成图像是否存在
            if not os.path.exists(generated_path):
                print(f"Warning: {filename} not found in generated directory")
                continue
                
            # 读取图像
            real_image = cv2.imread(real_path, cv2.IMREAD_GRAYSCALE)
            generated_image = cv2.imread(generated_path, cv2.IMREAD_GRAYSCALE)
            
            # 确保图像大小一致
            if real_image.shape != generated_image.shape:
                generated_image = cv2.resize(generated_image, (real_image.shape[1], real_image.shape[0]))
            
            # 计算指标
            ssim_score = ssim(real_image, generated_image, data_range=real_image.max() - real_image.min())
            psnr_score = psnr(real_image, generated_image, data_range=real_image.max() - real_image.min())
            lncc_score = calculate_lncc(real_image, generated_image)
            
            # 存储到对应类别
            class_metrics[class_name]['ssim'].append(ssim_score)
            class_metrics[class_name]['psnr'].append(psnr_score)
            class_metrics[class_name]['lncc'].append(lncc_score)
    
    # 计算每个类别的平均指标
    results = {}
    for class_name, metrics in class_metrics.items():
        results[class_name] = {
            'avg_ssim': np.mean(metrics['ssim']),
            'avg_psnr': np.mean(metrics['psnr']),
            'avg_lncc': np.mean(metrics['lncc']),
            'num_samples': len(metrics['ssim'])  # 该类别的样本数量
        }
    
    return results

# 定义真实图像和生成图像的目录
real_dir = "/home/user9/us_test/high_quality"
generated_dir = "/home/user9/us_test/result"

# 计算按类别分组的指标
class_results = calculate_metrics_by_class(real_dir, generated_dir)

# 输出结果
for class_name, metrics in class_results.items():
    print(f"\nClass: {class_name}")
    #print(f"  Number of samples: {metrics['num_samples']}")
    print(f"  Average SSIM: {metrics['avg_ssim']:.4f}")
    print(f"  Average PSNR: {metrics['avg_psnr']:.4f}")
    print(f"  Average LNCC: {metrics['avg_lncc']:.4f}")

# 计算全局平均指标
global_ssim = np.mean([m['avg_ssim'] for m in class_results.values()])
global_psnr = np.mean([m['avg_psnr'] for m in class_results.values()])
global_lncc = np.mean([m['avg_lncc'] for m in class_results.values()])

print("\nGlobal Averages:")
print(f"  Average SSIM: {global_ssim:.4f}")
print(f"  Average PSNR: {global_psnr:.4f}")
print(f"  Average LNCC: {global_lncc:.4f}")

total_samples = sum(m['num_samples'] for m in class_results.values())
global_ssim_weighted = sum(m['avg_ssim'] * m['num_samples'] for m in class_results.values()) / total_samples
global_psnr_weighted = sum(m['avg_psnr'] * m['num_samples'] for m in class_results.values()) / total_samples
global_lncc_weighted = sum(m['avg_lncc'] * m['num_samples'] for m in class_results.values()) / total_samples

print("\nWeighted Global Averages (by sample size):")
print(f"  Average SSIM: {global_ssim_weighted:.4f}")
print(f"  Average PSNR: {global_psnr_weighted:.4f}")
print(f"  Average LNCC: {global_lncc_weighted:.4f}")


Class: thyroid
  Average SSIM: 0.1418
  Average PSNR: 13.8462
  Average LNCC: 0.0013

Class: breast
  Average SSIM: 0.0987
  Average PSNR: 12.6931
  Average LNCC: 0.0116

Class: carotid
  Average SSIM: 0.1300
  Average PSNR: 12.7912
  Average LNCC: 0.0022

Class: kidney
  Average SSIM: 0.3823
  Average PSNR: 15.7534
  Average LNCC: 0.0043

Class: liver
  Average SSIM: 0.3749
  Average PSNR: 15.8301
  Average LNCC: 0.0061

Global Averages:
  Average SSIM: 0.2255
  Average PSNR: 14.1828
  Average LNCC: 0.0051

Weighted Global Averages (by sample size):
  Average SSIM: 0.1913
  Average PSNR: 13.8402
  Average LNCC: 0.0046


In [1]:
import os
import shutil

# 源目录（包含多个子目录，每个子目录下有 low_quality）
src_root = "/home/user9/us_train"
# 目标目录（保存所有 low_quality 图片）
dst_dir = "/home/user9/us_train_low_quality_images"

# 创建目标目录
os.makedirs(dst_dir, exist_ok=True)

# 遍历所有子目录
for root, dirs, files in os.walk(src_root):
    if os.path.basename(root) == "low_quality":
        # 复制 low_quality 目录下的所有图片
        for file in files:
            if file.lower().endswith(('.png', '.jpg', '.jpeg')):
                src_path = os.path.join(root, file)
                dst_path = os.path.join(dst_dir, file)
                # 避免文件名冲突（如果重复则添加前缀）
                if os.path.exists(dst_path):
                    base, ext = os.path.splitext(file)
                    dst_path = os.path.join(dst_dir, f"{base}_dup{ext}")
                shutil.copy2(src_path, dst_path)
print("所有图片已复制到:", dst_dir)

所有图片已复制到: /home/user9/us_train_low_quality_images


In [ ]:
from data import create_dataset, Unaligned2Dataset

dataset = Unaligned2Dataset()
